In [1]:
# temporarly add to path for testing purposes before package creation
import sys
sys.path.append("../../")

In [2]:
from phdtools.ASEtools import asetools as at

# ASE tools for handling trajs

We will use this small trajectory of Li-ion electrolytes as a tutorial.
This example showcases some key functionalities of the pkg.

In [3]:
!ls ../data/

traj_2.1_0-100-1.xyz


### _Blind_ Universe definition

The _blind_ means that we do not set any project specific variables, but only the minimum required to init the ASE Universe.

In [4]:
blind_project_dict = dict(
    projectName = "example_1_LiElectrolytes",
    trajPath = "../data/traj_2.1_0-100-1.xyz",
)

In [5]:
blind_universe = at.Universe(**blind_project_dict)

Gathering the Universe ...



Total atoms: 8402
Atom types: ['C' 'F' 'H' 'Li' 'O' 'P']

!!! Warning: `rcutCorrection` not set !!!
Default values will be used, this might cause artifacts in the molecules detection.
rcutCorrection = {'C': 1.0, 'F': 1.0, 'H': 1.0, 'Li': 1.0, 'O': 1.0, 'P': 1.0}

Searching for molecules in the system ...

!!! Warning: `moleculeNames` not set !!!
Default molecules names will be used

Uniques molecules found: 29
Molecules found: {'C10H16F6LiO9P': 'mol1', 'C11H20F6LiO9P': 'mol2', 'C11H20LiO9': 'mol3', 'C12H24F6LiO9P': 'mol4', 'C13H20LiO12': 'mol5', 'C14H24F12Li2O12P2': 'mol6', 'C14H24F18Li2O12P3': 'mol7', 'C14H24F6LiO12P': 'mol8', 'C14H24LiO12': 'mol9', 'C15H28F12Li2O12P2': 'mol10', 'C15H28F6LiO12P': 'mol11', 'C15H28LiO12': 'mol12', 'C16H32F12Li2O12P2': 'mol13', 'C16H32LiO12': 'mol14', 'C18H32F12Li2O15P2': 'mol15', 'C19H36F12Li2O15P2': 'mol16', 'C20H40F12Li2O15P2': 'mol17', 'C20H40F18Li2O15P3': 'mol18', 'C20H40F24Li3O15P4': 'mol19', 'C21H36F6Li2O18P': 'mol20', 'C22H40F6Li2O18P': 'mol21', 

In this case since we did not set the `rcutCorrection` it caused some artifacts in the molecule detection: some solevation shells are considered as whole molecules.
This is not convenient if we want to follow the Li during the trajectory, as the definition of the molecules changes constantly during the atoms evolution.

In [6]:
print(f"Default: {blind_universe.rcutCorrection}")

Default: {'C': 1.0, 'F': 1.0, 'H': 1.0, 'Li': 1.0, 'O': 1.0, 'P': 1.0}


To solve the problem we need to define a tailored `rcutCorrection` and repeat the init of the Universe.

In [7]:
rcut_dict = {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

In [8]:
blind_universe.set_rcut_correction(rcut_dict=rcut_dict)
print(f"Modified: {blind_universe.rcutCorrection}")

Updated rcutCorrection = {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

Modified: {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}


In [9]:
blind_universe.find_molecs(mol_name=None)
print(f"Default molecule names: {blind_universe.moleculeNames}")

Uniques molecules found: 4
Molecules found: {'C3H4O3': 'mol1', 'C4H8O3': 'mol2', 'F6P': 'mol3', 'Li': 'mol4'}
1.4265s

Default molecule names: ['mol1', 'mol2', 'mol3', 'mol4']


By default `find_molecs()` uses standard numbered names "molXXX" to address the molecules, but these can be specified as well.

In [10]:
mol_names_list = ['EC', 'EMC', 'PF6', 'Li']

blind_universe.set_mol_names(mol_name=mol_names_list)
print(f"Renamed molecules: {blind_universe.moleculeFormulas}\n"
      f"{blind_universe.moleculeNames}")

Renamed molecules: {'C3H4O3': 'EC', 'C4H8O3': 'EMC', 'F6P': 'PF6', 'Li': 'Li'}
['EC', 'EMC', 'PF6', 'Li']


Alternatively it is possible to define them directly in the `find_molecs()` function.

In [11]:
blind_universe.find_molecs(mol_name=mol_names_list)
print(f"Molecules names: {blind_universe.moleculeFormulas}\n"
      f"{blind_universe.moleculeNames}")

Uniques molecules found: 4
Molecules found: {'C3H4O3': 'EC', 'C4H8O3': 'EMC', 'F6P': 'PF6', 'Li': 'Li'}
1.4533s

Molecules names: {'C3H4O3': 'EC', 'C4H8O3': 'EMC', 'F6P': 'PF6', 'Li': 'Li'}
['EC', 'EMC', 'PF6', 'Li']


### _Complete_ Universe definition

The _complete_ means that we do set any project specific variables when init the ASE Universe.

In [12]:
complete_project_dict = dict(
    projectName = "example_1_LiElectrolytes",
    trajPath = "../data/traj_2.1_0-100-1.xyz",
    rcutCorrection = rcut_dict,
    moleculeNames = mol_names_list
)

In [13]:
complete_universe = at.Universe(**complete_project_dict)

Gathering the Universe ...

Total atoms: 8402
Atom types: ['C' 'F' 'H' 'Li' 'O' 'P']

rcut correction: {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

Searching for molecules in the system ...

['EC', 'EMC', 'PF6', 'Li']
Uniques molecules found: 4
Molecules found: {'C3H4O3': 'EC', 'C4H8O3': 'EMC', 'F6P': 'PF6', 'Li': 'Li'}
1.2648s

Computing MolIDs ...
1.3113s

Computing MolSymbols ...
Total numner of molecules: 777
0.0489s

<end>
2.7684s



In [14]:
complete_universe.projectDictionary

{'projectName': 'example_1_LiElectrolytes',
 'trajPath': '../data/traj_2.1_0-100-1.xyz',
 'rcutCorrection': {'H': 1.0,
  'C': 1.0,
  'O': 1.0,
  'Li': 0.1,
  'P': 1.0,
  'F': 1.0},
 'moleculeNames': ['EC', 'EMC', 'PF6', 'Li'],
 'moleculeFormulas': {'C3H4O3': 'EC',
  'C4H8O3': 'EMC',
  'F6P': 'PF6',
  'Li': 'Li'},
 'molIDs': array([  0,   0,   0, ..., 774, 775, 776], dtype=int32),
 'molSym': ['EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',
  'EC',

## Handling the Ase Trajectory

In [4]:
rcut_dict = {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}
mol_names_list = ['EC', 'EMC', 'PF6', 'Li']

TODO

In [5]:
project_dict = dict(
    projectName = "example_1_LiElectrolytes",
    trajPath = "../data/traj_2.1_0-100-1.xyz",
    rcutCorrection = rcut_dict,
    moleculeNames = mol_names_list,
    frameRange = (0,30,2)
)

In [6]:
aseU = at.ASEtraj(**project_dict)

Gathering the Universe ...

Total atoms: 8402
Atom types: ['C' 'F' 'H' 'Li' 'O' 'P']

rcut correction: {'H': 1.0, 'C': 1.0, 'O': 1.0, 'Li': 0.1, 'P': 1.0, 'F': 1.0}

Searching for molecules in the system ...

['EC', 'EMC', 'PF6', 'Li']
Uniques molecules found: 4
Molecules found: {'C3H4O3': 'EC', 'C4H8O3': 'EMC', 'F6P': 'PF6', 'Li': 'Li'}
1.2139s

Computing MolIDs ...
1.1806s

Computing MolSymbols ...
Total numner of molecules: 777
0.0470s

<end>
2.6070s



TODO

In [7]:
aseU.frameRange = (0,1)

In [8]:
aseU.frameRange

(0, 1)

In [9]:
ase_db = aseU._read

Reading traj:
Begin: 0 | End: 1 | Stride: 1


In [10]:
ase_db[0].numbers

array([6, 6, 8, ..., 3, 3, 3])

In [11]:
from phdtools.ASEtools import atomstools

In [12]:
atomstools.ZnumberShift(ase_db[0].numbers, aseU.molSym, aseU.molIDs, ('EC', [6,8]))

array([21, 21, 23, ...,  3,  3,  3])